# **📐Description**
- 이미지를 정방향으로 회전시키는 CV 모델을 학습하는 코드입니다.
- UPSTAGE AI LAB 문서분류 대회에서 Test 데이터셋을 정방향으로 돌리기 위해서 작성되었으며 따라서 해당대회의 Test 데이터셋에 맞춘 Augmentation이 적용돼 있음을 알려드립니다.
- 해당 코드는 https://github.com/d4nst/RotNet 의 코드를 참고하여 pytorch로 재구현했습니다.

## Contents
- Import Library & Define Functions
- Data Process
- Hyper-parameters
- Augmenation
- Data Definition
- Train Model
- Test Inference

## 1 Import Library & Define Functions

In [33]:
import os
import sys

import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import timm
import torch
import torch.nn as nn
from torch import optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from transformers import get_scheduler

import os
import wget
import zipfile

In [34]:
# 정규화된 이미지를 원래 범위로 되돌리기(EDA용)
def denormalize_image1(normalized_image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    denormalized_image = normalized_image * np.array(std)[:, None, None] + np.array(mean)[:, None, None]
    denormalized_image = np.clip(denormalized_image, 0, 1)
    
    if isinstance(denormalized_image, torch.Tensor):
        denormalized_image = (denormalized_image * 255).byte()
    else:
        denormalized_image = (denormalized_image * 255).astype(np.uint8)
        
    return denormalized_image

In [35]:
# Tensor 이미지 시각화용(EDA용)
def show_img(tensor, denormalize=False):
    if denormalize:
        tensor = denormalize_image1(tensor)
    ten_iamge = tensor.transpose(0, 2).transpose(0, 1)
    # ten_iamge = cv2.cvtColor(np.array(ten_iamge), cv2.COLOR_BGR2RGB)
    plt.imshow(ten_iamge)

In [36]:
def angle_difference(x, y):
    """
    Calculate minimum difference between two angles.
    """
    return 180 - abs(abs(x - y) - 180)

def angle_error(y_pred, y_true):
    """
    Calculate the mean difference between the true angles
    and the predicted angles. Each angle is represented
    as a one-hot encoded vector.
    """
    true_angle = y_true
    pred_angle = torch.argmax(y_pred, dim=1)
    diff = angle_difference(true_angle.float(), pred_angle.float())
    return torch.mean(torch.abs(diff.float()))


In [37]:
# 이미지 회전코드(학습용)
def rotate(image, angle):
    height, width = image.shape[:2]
    center = (width / 2, height / 2)

    # 회전 변환 행렬 생성
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale=1.0)

    # 이미지 회전
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    
    return rotated_image
    

In [38]:
# 이미지 손실을 최소화하는 이미지 회전코드(결과 이미지 저장용)
def rotate2(image, angle):
    
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    
    # 회전 변환 행렬 계산
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 새로운 이미지의 크기 계산
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    
    # 회전 중심점 조정
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]
    
    # 이미지 회전
    rotated_image = cv2.warpAffine(image, rotation_matrix, (new_w, new_h), flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))
    
    return rotated_image

### 1-1 Dataset 정의

In [39]:
class RotNetDataset(Dataset):
    def __init__(self, df, path, transform=None, rotate=None, crop=None):
        self.df = df.values
        self.path = path
        self.rotate = rotate
        self.transform = transform
        self.crop = crop

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, _ = self.df[idx]
        img = cv2.imread(os.path.join(self.path, name))
        
        if self.transform:
            img = self.transform(image=img)['image']
        
        angle = 0
        chance = random.random()
        if self.rotate:
            if chance < 0.1:
                angle = 90
                img = self.rotate(img, angle)
            elif chance < 0.2:
                angle = 180
                img = self.rotate(img, angle)
            elif chance < 0.3:
                angle = 270
                img = self.rotate(img, angle)
            elif chance < 0.4:
                angle = 0
            else:
                angle = np.random.randint(360)
                img = self.rotate(img, angle)
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.crop:
            img = self.crop(image=img)['image']
            
        transf = A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255),
            ToTensorV2(),
        ])
        
        img = transf(image=img)['image']

        return img, angle


## 2 Data Process
- 원래 Document Classification Task용 데이터이기 때문에 전처리가 어색한 부분이 있을 수 있습니다.

In [40]:
# 클래스 이름 데이터
meta_kr_df = pd.read_csv("../data/meta_kr.csv", encoding='euc-kr')

# 이미지 filename과 target
df = pd.read_csv("../data/train.csv")

# 이상데이터 처리(잘못된 라벨링, 중복 데이터)
df.loc[df['ID'] == '45f0d2dfc7e47c03.jpg', 'target'] = 7
df = df[df['ID'] != 'aec62dced7af97cd.jpg']
df = df[df['ID'] != 'c5182ab809478f12.jpg']
df.loc[df['ID'] == '8646f2c3280a4f49.jpg', 'target'] = 3
df = df[df['ID'] != '1ec14a14bbe633db.jpg']

# 각도 이상 데이터 제외
df = df[df['ID'] != '2552a6625015ace2.jpg']
df = df[df['ID'] != '7ce5e5dc4eb34620.jpg']
df = df[df['ID'] != 'edbf627d7217a4af.jpg']

# EDA를 위해 meta 데이터 병합
df = df.merge(meta_kr_df, how='left', on='target')
df['path'] = "../data/train/" + df['ID']

df_test = pd.read_csv("../data/sample_submission.csv")
df_test['path'] = "../data/test/" + df_test['ID']

### 2-1 Train/Valid 분활

In [41]:
# Stratified K-Fold 를 이용해서 분활
temp_X = df.drop(columns=['target'])
temp_y = df['target']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(temp_X, temp_y):
    new_df = df.iloc[train_index]
    val_df = df.iloc[test_index]
    break

In [42]:
# 불균형 해결하기
oversample_cols = ['resume', 'statement_of_opinion', 'application_for_payment_of_pregnancy_medical_expenses']

def oversampling(df, cols):
    class_num = df['class_name'].value_counts().max()
    temp = df[df['class_name'] == 'resume']
    samples = temp.sample(class_num - temp.shape[0])
    df = pd.concat([df, samples], axis=0)

    temp = df[df['class_name'] == 'statement_of_opinion']
    df = pd.concat([df, temp], axis=0)

    temp = df[df['class_name'] == 'application_for_payment_of_pregnancy_medical_expenses']
    df = pd.concat([df, temp], axis=0)
    
    return df

In [43]:
new_df = oversampling(new_df, oversample_cols)
val_df = oversampling(val_df, oversample_cols)

### 2-2 Data 선별
- 회전된 이미지가 많이 포함된 Test 셋에 대해서 정방향으로 이미지를 돌린후 제목을 Crop하고 싶은 것이기 때문에 일반적인 문서 이미지와 상이한 데이터들은 노이즈로 간주하고 제외해줍니다.

In [44]:
drop_class = ['account_number', 'car_dashboard', 'driver_lisence', 'national_id_card', 'passport', 'vehicle_registration_plate']

sel_tr_df = new_df[~new_df['class_name'].isin(drop_class)]
sel_val_df = val_df[~val_df['class_name'].isin(drop_class)]
sel_meta_kr_df = meta_kr_df[~meta_kr_df['class_name'].isin(drop_class)]

## 3 Hyper-parameters

In [45]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'timm/caformer_s18.sail_in22k_ft_in1k_384' 

# training config
img_size = 384
LR = 1e-4
EPOCHS = 100
BATCH_SIZE = 32
PATIENCE = 25
num_workers = 0

model_path = '../model/caformer/'
FILENAME = 'caformer_s18_sail_in22k_ft_in1k_384_rot.pth'

## 4 Augmenation

### 4-1 Albumentations

In [46]:
# 원본 데이터셋에 적용될 Albumentations
trn_transform = A.Compose([
    A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
    A.OneOf([
        A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.6),
        A.ChannelShuffle(p=0.4),
    ], p=0.2),
    
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
    A.OneOf([A.MotionBlur(p=0.3), A.MedianBlur(blur_limit=3, p=0.3), A.Blur(blur_limit=3, p=0.4),], p=0.2),
    A.GaussNoise(var_limit=(30.0, 250.0), mean=0, per_channel=True, p=0.2),
    A.RandomBrightnessContrast (brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), brightness_by_max=True, p=0.2),
    A.OneOf([A.OpticalDistortion(p=0.5), A.PiecewiseAffine(scale=(0.01, 0.03), p=0.5)], p=0.15),
    A.HorizontalFlip(p=0.5),
])

# augraphys가 적용된 데이터셋에 적용할 Albumentations
# GaussNoise외에는 trn_transform과 동일
trn_transform02 = A.Compose([
    A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
    A.OneOf([
        A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.6),
        A.ChannelShuffle(p=0.4),
    ], p=0.2),
    
    A.OneOf([A.MotionBlur(p=0.3), A.MedianBlur(blur_limit=3, p=0.3), A.Blur(blur_limit=3, p=0.4),], p=0.2),
    A.RandomBrightnessContrast (brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), brightness_by_max=True, p=0.2),
    A.OneOf([A.OpticalDistortion(p=0.5), A.PiecewiseAffine(scale=(0.01, 0.03), p=0.5)], p=0.15),
    A.HorizontalFlip(p=0.5),
])

# 이미지를 회전시킨 이후에 Crop할 예정이기 때문에 미리 정의해 둡니다.
# 이미지를 미리 Crop하고 회전시킬 경우 이미지의 외각을 보고 각도를 예측할 수 있기 때문에 미리 Crop 하지 않습니다.
random_crop = A.Compose([
    A.RandomCrop (img_size, img_size),
])

center_crop = A.Compose([
    A.CenterCrop (img_size, img_size),
])


# test 셋에 대해서 TTA를 적용하기 위한 Augmentation
tst_transform = A.Compose([
    A.OneOf([
        A.Compose([
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0, p=0.6, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            A.CenterCrop (img_size, img_size, p=1.0),
        ], p=0.6),
        A.Compose([
            A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            A.RandomCrop(img_size, img_size),
        ], p=0.4)
    ], p=1.0),
])

## 5 Data Definition

### 5-1 Dataset 정의

In [47]:
# 원본 train + Albumentations
trn_dataset = RotNetDataset(
    sel_tr_df[['ID', 'target']],
    "../data/train/",
    transform=trn_transform,
    rotate=rotate,
    crop=center_crop
)

# augraphys train + Albumentations
# 200 에폭 분량의 데이터셋 정의
aug_trn_dataset_list = []
for i in range(0, 200):
    folder_path = f"../data/aug/train{str(i)}/"
    aug_trn_dataset = RotNetDataset(new_df[['ID', 'target']], folder_path, transform=trn_transform02, rotate=rotate, crop=random_crop)
    aug_trn_dataset_list.append(aug_trn_dataset)


# 원본 valid + Albumentations
val_dataset = RotNetDataset(
    sel_val_df[['ID', 'target']],
    "../data/train/",
    transform=trn_transform,
    rotate=rotate,
    crop=center_crop
)

# augraphys valid + Albumentations
# 200 에폭 분량의 데이터셋 정의
aug_val_dataset_list = []
for i in range(0, 200):
    folder_path = f"../data/aug/train{str(i)}/"
    aug_val_dataset = RotNetDataset(val_df[['ID', 'target']], folder_path, transform=trn_transform02, rotate=rotate, crop=random_crop)
    aug_val_dataset_list.append(aug_val_dataset)


# RotNet으로 회전시킨 test셋을 불러와서 Dataset 구성
tst_dataset = RotNetDataset(
    df_test[['ID', 'target']],
    "../data/test/",
    transform=tst_transform,
    crop=None
)

print(len(trn_dataset), len(val_dataset), len(tst_dataset))
print(len(aug_trn_dataset_list), len(aug_val_dataset_list))

870 218 3140
200 200


### 5-2 Dataset EDA
- 데이터에 augmenation이 잘 적용됐는지 확인용

In [ ]:
ran = random.randint(0, len(tst_dataset))
# ran=495
image, target = tst_dataset[ran]

plt.figure(figsize=(10, 8))
plt.title(f"index: {ran}, target: {target}")
show_img(image, True)

In [ ]:
ran = random.randint(0, len(trn_dataset))
image, target = trn_dataset[ran]

plt.figure(figsize=(10, 8))
plt.title(f"index: {ran}, target: {target}")
show_img(image, True)

In [50]:
# idx = random.randint(0, len(aug_val_dataset_list[0]))
# # idx = 2

# plt.figure(figsize=(10, 10))
# plt.title(idx)
# show_img(aug_val_dataset_list[0][idx][0], True)

### 5-3 DataLoader 정의

In [51]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

print(len(trn_loader), len(val_loader), len(tst_loader))

28 7 99


## 6 Train Model

In [52]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=360
).to(device)
loss_fn = nn.CrossEntropyLoss()

In [53]:
# 학습코드
def training(model, dataloader, optimizer, loss_fn, scheduler, device, epoch, num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    
    train_loss = 0.0
    trn_angle_loss = 0.0
    preds_list = []
    targets_list = []

    tbar = tqdm(dataloader)
    for idx, (image, targets) in enumerate(tbar):
        image = image.to(device)
        targets = targets.to(device)
        
        # 순전파
        model.zero_grad(set_to_none=True)
        preds = model(image)
        loss = loss_fn(preds, targets)
        angle_loss = angle_error(preds, targets)
        
        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # 손실과 정확도 계산
        train_loss += loss.item()
        trn_angle_loss += angle_loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.8f}, Train angle loss: {angle_loss:.8f}")
        
    
    # 에폭별 학습 결과 출력
    train_loss = train_loss / len(dataloader)
    trn_angle_loss = trn_angle_loss / len(dataloader)
    
    ret = {
        "train_loss": train_loss,
        "trn_angle_loss": trn_angle_loss 
    }

    return model, ret

# 평가코드
def evaluation(model, dataloader, loss_fn, device, epoch, num_epochs):
    model.eval()  # 모델을 평가 모드로 설정
    valid_loss = 0.0
    valid_angle_loss = 0.0
    preds_list = []
    targets_list = []

    with torch.no_grad(): # model의 업데이트 막기
        tbar = tqdm(dataloader)
        for idx, (image, targets) in enumerate(tbar):
            image = image.to(device)
            targets = targets.to(device)

            # 순전파
            model.zero_grad(set_to_none=True)
            preds = model(image)
            loss = loss_fn(preds, targets)
            angle_loss = angle_error(preds, targets)

            # 손실과 정확도 계산
            valid_loss += loss.item()
            valid_angle_loss += angle_loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Valid Loss: {loss.item():.8f}, Valid angle loss: {angle_loss:.8f}")
            
    # 에폭별 학습 결과 출력
    valid_loss = valid_loss / len(dataloader)
    valid_angle_loss = valid_angle_loss / len(dataloader)
    
    ret = {
        "valid_loss": valid_loss,
        "valid_angle_loss": valid_angle_loss 
    }

    return model, ret

# 학습 루프
def training_loop(model, train_dataloader, valid_dataloader, trn_dataset, val_dataset, train_dataset_list, val_dataset_list, loss_fn, optimizer, scheduler, device, num_epochs, patience, filename):
    best_valid_loss = float('inf')  # 가장 좋은 validation loss를 저장
    early_stop_counter = 0  # 카운터
    loss_fn = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        # 매 에폭마다 다르게 Augraphys가 적용된 데이터 불러오기
        data_idx = epoch % 200
        T_dataset = ConcatDataset([trn_dataset, train_dataset_list[data_idx]])
        V_dataset = ConcatDataset([val_dataset, val_dataset_list[data_idx]])
        
        train_dataloader = DataLoader(T_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=False)
        valid_dataloader = DataLoader(V_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True, drop_last=False)
        print(f"dataset{data_idx}")
        
        model, train_ret = training(model, train_dataloader, optimizer, loss_fn, scheduler, device, epoch, num_epochs)
        model, valid_ret = evaluation(model, valid_dataloader, loss_fn, device, epoch, num_epochs)
        
        # validation loss가 감소하면 모델 저장 및 카운터 리셋
        if valid_ret['valid_loss'] < best_valid_loss:
            best_valid_loss = valid_ret['valid_loss']
            torch.save(model.state_dict(), model_path+filename)
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        print(f"Epoch [{epoch + 1}/{num_epochs}], T_Train Loss: {train_ret['train_loss']:.8f}, T_Train Angle Loss: {train_ret['trn_angle_loss']:.8f}")
        print(f"Epoch [{epoch + 1}/{num_epochs}], T_Valid Loss: {valid_ret['valid_loss']:.8f}, T_Valid Angle Loss: {valid_ret['valid_angle_loss']:.8f}")

        # early_stopping
        if early_stop_counter >= patience:
            print("Early stopping")
            break
            
    return model, best_valid_loss

In [48]:
# 모델 전체 fine tuning
model.to(device)
num_epochs = EPOCHS
lr = LR
patience = PATIENCE
filename = FILENAME

# optimizer = optim.RMSprop(model.parameters(), lr=LR, alpha=0.9)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05)
# optimizer = optim.Adam(model.parameters(), lr=LR)

# scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=LR, step_size_up=10, step_size_down=20, mode='triangular')

num_training_steps = num_epochs * len(trn_loader) * 2
scheduler = get_scheduler(
    name='cosine', optimizer=optimizer, 
    num_warmup_steps=0, 
    num_training_steps=num_training_steps
)

model, best_valid_loss = training_loop(model, trn_loader, val_loader, trn_dataset, val_dataset, aug_trn_dataset_list, aug_val_dataset_list, loss_fn, optimizer, scheduler, device, num_epochs, patience, filename)
print(f'Best valid loss : {best_valid_loss:5f}')

dataset0


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [1/100], Train Loss: 4.47293091, Train angle loss: 85.55555725: 100%|██████████| 70/70 [01:45<00:00,  1.51s/it] 
Epoch [1/100], Valid Loss: 4.42137957, Valid angle loss: 116.91667175: 100%|██████████| 18/18 [00:25<00:00,  1.43s/it]


Epoch [1/100], T_Train Loss: 4.19392953, T_Train Angle Loss: 92.69722225
Epoch [1/100], T_Valid Loss: 3.86324796, T_Valid Angle Loss: 87.19675954
dataset1


Epoch [2/100], Train Loss: 3.58662510, Train angle loss: 100.44444275: 100%|██████████| 70/70 [01:39<00:00,  1.42s/it]
Epoch [2/100], Valid Loss: 2.76875043, Valid angle loss: 103.91667175: 100%|██████████| 18/18 [00:20<00:00,  1.13s/it]


...

Epoch [98/100], T_Train Loss: 0.28012703, T_Train Angle Loss: 1.14246032
Epoch [98/100], T_Valid Loss: 0.29361674, T_Valid Angle Loss: 2.86400463
dataset98


Epoch [99/100], Train Loss: 0.20118208, Train angle loss: 0.00000000: 100%|██████████| 70/70 [01:31<00:00,  1.31s/it] 
Epoch [99/100], Valid Loss: 0.18563353, Valid angle loss: 0.08333334: 100%|██████████| 18/18 [00:18<00:00,  1.02s/it]


Epoch [99/100], T_Train Loss: 0.29487926, T_Train Angle Loss: 1.62276786
Epoch [99/100], T_Valid Loss: 0.28654114, T_Valid Angle Loss: 1.30324074
dataset99


Epoch [100/100], Train Loss: 0.78237033, Train angle loss: 0.55555558: 100%|██████████| 70/70 [01:45<00:00,  1.51s/it]
Epoch [100/100], Valid Loss: 0.14517120, Valid angle loss: 0.08333334: 100%|██████████| 18/18 [00:21<00:00,  1.18s/it]

Epoch [100/100], T_Train Loss: 0.28754946, T_Train Angle Loss: 1.16507937
Epoch [100/100], T_Valid Loss: 0.31895255, T_Valid Angle Loss: 1.81192130
Best valid loss : 0.245228


## 7 Test Inference

In [54]:
def infer_eval(model, dataloader, device, is_test):
    model.eval()  # 모델을 평가 모드로 설정
    valid_loss = 0.0
    valid_angle_loss = 0.0
    preds_list = []
    targets_list = []
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad(): # model의 업데이트 막기
        tbar = tqdm(dataloader)
        for idx, (image, targets) in enumerate(tbar):
            image = image.to(device)
            targets = targets.to(device)

            # 순전파
            model.zero_grad(set_to_none=True)
            preds = model(image)
            if not is_test:
                loss = loss_fn(preds, targets)
                angle_loss = angle_error(preds, targets)

            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())
            
            # 손실과 정확도 계산
            if not is_test:
                valid_loss += loss.item()
                valid_angle_loss += angle_loss.item()
                tbar.set_description(f"Valid Loss: {loss.item():.6f}, Valid angle loss: {angle_loss:.6f}")

    # 에폭별 학습 결과 출력
    if not is_test:   
        valid_loss = valid_loss / len(dataloader)
        valid_angle_loss = valid_angle_loss / len(dataloader)
        print(f"Valid Loss: {valid_loss:.6f}, Valid Angle Loss: {valid_angle_loss:.6f}")

    return preds_list, targets_list

In [56]:
# 모델 정의
model = timm.create_model(
    model_name,
    num_classes=360
).to(device)

# 저장된 가중치 불러오기
model.load_state_dict(torch.load(model_path+FILENAME))

<All keys matched successfully>

#### 7-1 TTA(Test-Time Augmentation)

In [57]:
predictions = []
num_augmentations = 100

for i in range(num_augmentations):
    preds_list, targets_list = infer_eval(model, tst_loader, device, True)
    predictions.append(preds_list)

preds = np.median(predictions, axis=0)

100%|██████████| 99/99 [00:34<00:00,  2.89it/s]


In [60]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds
pred_df['path'] = '../data/test/' + pred_df['ID']

class_pred = pd.read_csv('../scripts/output_best.csv')
pred_df['pred_class']=  class_pred['target']

temp = meta_kr_df.copy()
temp.columns = ['class_num', 'class_name', 'kr']
pred_df = pred_df.merge(temp, how='left', left_on='pred_class', right_on='class_num')
pred_df.drop(columns='class_num', inplace=True)

#### 7-2 Test 예측 결과 Inferance

In [ ]:
samples = pred_df.sample(5)
plt.figure(figsize=(20, 40))

for idx in range(len(samples)):
    sample = samples.iloc[idx]
    
    plt.subplot(5, 2, idx*2+1)
    plt.title("Original")
    image = cv2.cvtColor(cv2.imread(sample['path']), cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    
    plt.subplot(5, 2, idx*2+2)
    plt.title(f"Target: {sample['target']}")
    
    rot_image = rotate2(image, -sample['target'])
    plt.imshow(rot_image)
    

#### 7-2 이미지 저장

In [74]:
# 예측결과를 기반으로 이미지를 회전시켜서 저장
for idx in range(len(pred_df)):
    filename, target, path = pred_df.iloc[idx][['ID', 'target', 'path']]

    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    rot_image = rotate2(image, -target)
    save_path = os.path.join('../data/test_rot_cafomer01/', filename)
    
    cv2.imwrite(save_path, rot_image) 

print(len(pred_df), 'files is saved')

3140 files is saved
